# Fresh take

Trying to get this right.  2024/02/18

In [ ]:
import os
import shutil
import importlib
import NewDev as ND

importlib.reload(ND)     

# fresh database
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass
# ND.database(projectPath=projectPath)

ND.siteInventory(projectPath=projectPath,siteID='SCL',
#   Name: Fish Island
#   PI: June Skeeter
  siteDescription = 'Eddy covariance station low center polygon permafrost peatland site in the Mackenzie River Delta',
#   stopDate: '2017-09-13'
#   landCoverType: Peatland
  latitude=69.3722778,
  longitude=-134.8810889,
#   startDate: '2017-06-21'
  subSites={'SSM':{
  'subsiteID':'SSM',
  'description':'Salix sedge meadow',
  'latitude':69.3722778,
  'longitude':-134.8810889}}
  )

ND.measurementInventory(projectPath=projectPath,measurementID='Flux',siteID='SCL',
  measurementDescription = 'High frequency (20 Hz) Eddy Covariance data',
  # fileType = 'TOBA',
  # frequency = '50ms',
  # latitude = 69.2264167,
  # longitude = -135.2519067,
  # startDate = '2024-07-20'
  )

t = ND.measurementInventory(projectPath=projectPath,measurementID='Profile',siteID='SCL',
  measurementDescription = 'Salix sedge meadow soil temperature profile',
  # fileType = 'TOBA',
  # frequency = '50ms',
  # latitude = 69.2264167,
  # longitude = -135.2519067,
  # startDate = '2024-07-20'
  )

t = ND.measurementInventory(projectPath=projectPath,measurementID='Profile',siteID='SCL',
  measurementDescription = 'Salix sedge meadow soil temperature profile',
  # fileType = 'TOBA',
  # frequency = '50ms',
  # latitude = 69.2264167,
  # longitude = -135.2519067,
  # startDate = '2024-07-20'
  )
# t.__dict__
t = ND.fileInventory(projectPath=projectPath,measurementID='Profile',siteID='SCL',subsiteID='WSM')
t = ND.fileInventory(projectPath=projectPath,measurementID='Profile',siteID='SCL',subsiteID='SSM')
# t = ND.fileInventory(projectPath=projectPath,measurementID='Flux',siteID='SCL')
t.inventory.keys()

confirm validation and filling procedures
setup to save full project metadata?
confirm validation and filling procedures
setup to save full project metadata?
confirm validation and filling procedures
setup to save full project metadata?
confirm validation and filling procedures
setup to save full project metadata?
SCL-Flux
?
Unsafe name:  SCL-Flux  
Converting to:  SCL_Flux
confirm validation and filling procedures
setup to save full project metadata?


dict_keys(['SCL_Flux'])

In [249]:
# import copy
# def emptyNest(seq):
#     def addVal(d,k,v):
#         d.setdefault(k,v)
#         return(d)
#     d = {}
#     for s in seq:
#         d = addVal({},s,d)
#     return(d)
# emptyNest(['a','b','c'])

import re

tst = 'some_random test-ojbject ahta.235.--?daf'
tst = re.sub('[^0-9a-zA-Z_]+','-', tst)
tst
                    # if self.__dataclass_fields__[k].repr and k != 'source':
                    #     self.__dict__[k] = re.sub('[^0-9a-zA-Z.]+',self.fillChar, self.__dict__[k])
                    # elif k.endswith('Date'):
                    #     self.__dict__[k] = re.sub('[^0-9a-zA-Z.]+',self.sepChar, self.__dict__[k])

'some_random-test-ojbject-ahta-235-daf'

In [232]:
l = 'abc'
import numpy as np
import random
# sum(l)
random.seed(1)
print(random.randint())

TypeError: Random.randint() missing 2 required positional arguments: 'a' and 'b'

## Setup an inventory

In [ ]:
import os
import yaml
import time
import shutil
import numpy as np
import pandas as pd
import importlib
import NewDev as ND
import helperFunctions as hf
importlib.reload(ND)     

# fresh database
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass
ND.databaseProject(projectPath)

# load templates
template = hf.loadDict(r'example_data\siteInventory.yml')
for siteID,values in template.items():
    ND.siteInventory(projectPath=projectPath,siteID=siteID,**values)

template = hf.loadDict(r'example_data\SCL\measurementInventory.yml')
for measurementID,values in template.items():
    ND.measurementInventory(projectPath=projectPath,measurementID=measurementID,siteID='SCL',**values)

source = r'example_data'
ND.fileInventory(projectPath=projectPath,source=source,siteID='SCL',measurementID='Profile_SSM',ext='csv',matchPattern='SSM')#,lookup=True)
ND.fileInventory(projectPath=projectPath,source=source,siteID='SCL',measurementID='Profile_WSM',ext='csv',matchPattern='WSM')#,lookup=True)

siteID
measurementInventory(measurementID=None)
None
siteID
measurementInventory(measurementID='Flux')
SCL
siteID
measurementInventory(measurementID='Metadata')
SCL
siteID
measurementInventory(measurementID='Meteorological')
SCL
siteID
measurementInventory(measurementID='Profile_SSM')
SCL
siteID
measurementInventory(measurementID='Profile_WSM')
SCL
siteID
fileInventory(source='c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\example_data')
SCL
measurementID
fileInventory(source='c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\example_data')
Profile_SSM
siteID
fileInventory(source='c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\example_data')
SCL
measurementID
fileInventory(source='c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\example_data')
Profile_WSM


fileInventory(source='c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\example_data')

In [ ]:
ND.fileInventory(projectPath=projectPath,siteID='SCL',measurementID='Profile_WSM',lookup=True).sessionMetadata
# a = None
# if a:
#     print('d')

siteID
fileInventory(source=None)
SCL
measurementID
fileInventory(source=None)
Profile_WSM


{}

## File Management

In [ ]:
import time
import rawDataFile as rd
importlib.reload(rd)
source = r'example_data'
T1 = time.time()

t = rd.fileInventory(projectPath=projectPath,source=source,siteID='SCL',measurementID='Profile_SSM',ext='csv',matchPattern='SSM',lookup=True)

t.inventory

{'c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\example_data': {'ext': 'csv',
  'matchPattern': ['SSM'],
  'excludePattern': [],
  'fileList': [['20240724\\20750528-SHSC.SSM.SGT.240720_240724readout.csv',
    False],
   ['20240914\\20750528-SHSC.SSM.SGT.240720_240913readout.csv', False]]}}

In [1]:
import pandas as pd
pd.date_range('2025-03-01','2025-03-02',freq='50ms')

DatetimeIndex([       '2025-03-01 00:00:00', '2025-03-01 00:00:00.050000',
               '2025-03-01 00:00:00.100000', '2025-03-01 00:00:00.150000',
               '2025-03-01 00:00:00.200000', '2025-03-01 00:00:00.250000',
               '2025-03-01 00:00:00.300000', '2025-03-01 00:00:00.350000',
               '2025-03-01 00:00:00.400000', '2025-03-01 00:00:00.450000',
               ...
               '2025-03-01 23:59:59.550000', '2025-03-01 23:59:59.600000',
               '2025-03-01 23:59:59.650000', '2025-03-01 23:59:59.700000',
               '2025-03-01 23:59:59.750000', '2025-03-01 23:59:59.800000',
               '2025-03-01 23:59:59.850000', '2025-03-01 23:59:59.900000',
               '2025-03-01 23:59:59.950000',        '2025-03-02 00:00:00'],
              dtype='datetime64[ns]', length=1728001, freq='50ms')

In [ ]:
# fn
import os
import rawDataFile as rawDataFile
importlib.reload(rawDataFile)
# fn = fi['SCL']['TS_Profile-WSM']['c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024']['sourceFiles'][0]
# fn = os.path.join('c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024',fn)
fn = r'c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024\\20240724\\20750527-SHSC.WSM.SGT.240720_240724readout.csv'
rd = rawDataFile.hoboCSV(projectPath=projectPath,source=fn,siteID='SCL',measurementType='TS_Profile',loggerID='WSM',
                         Metadata='test.yml')

with open('test.yml','w+') as f:
    yaml.safe_dump(rd.Metadata,f,sort_keys=False)

{'TS_1_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 5)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 5 cm depth', 'vertical': 1, 'horizontal': 1, 'replicate': 1}, 'TS_2_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 25)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 25 cm depth', 'vertical': 2, 'horizontal': 1, 'replicate': 1}, 'TS_3_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 50)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 50 cm depth', 'vertical': 3, 'horizontal': 1, 'replicate': 1}, 'TS_4_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 100)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temper

# Database

In [ ]:
importlib.reload(ND)

# projectPath = os.path.join(os.getcwd(),'test2')
# # if os.path.isdir(projectPath):
# #     shutil.rmtree(projectPath)

# db = pyDB.database(projectPath=projectPath)
# # db = pyDB.database(projectPath=projectPath,
# #     # siteID=['SCL'],
# #     # Years=[str(y) for y in range(2024,2025)],
# #     # verbose=False
# #     )
# # db.
n = ND.rawDatabaseImport(projectPath=projectPath,measurementID ='test',dataIn=rd.Data,metadataIn=rd.Metadata)

n

_ {'ignore': True, 'name_in': '#', 'unit_in': None, 'safe_name': '_', 'dtype': '<i8', 'variableDescription': None}
Date_Time_GMT_00_00 {'ignore': True, 'name_in': 'Date Time, GMT+00:00', 'unit_in': None, 'safe_name': 'Date_Time_GMT_00_00', 'dtype': '|O', 'variableDescription': None}
Temp_C_5cm {'ignore': False, 'name_in': 'Temp, °C (5cm', 'unit_in': None, 'safe_name': 'Temp_C_5cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_25cm {'ignore': False, 'name_in': 'Temp, °C (25cm', 'unit_in': None, 'safe_name': 'Temp_C_25cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_50cm {'ignore': False, 'name_in': 'Temp, °C (50cm', 'unit_in': None, 'safe_name': 'Temp_C_50cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_100cm {'ignore': False, 'name_in': 'Temp, °C (100cm', 'unit_in': None, 'safe_name': 'Temp_C_100cm', 'dtype': '<f8', 'variableDescription': None}


rawDatabaseImport(projectPath='c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\test', overwrite=False, verbose=True, logFile='Creating Database: 2025-03-01 21:22:42\n', metadataFile={'defaultFormat': {'data': {'dtype': 'float32', 'info': 'data are stored as 32-bit floating point numbers'}, 'POSIX_timestamp': {'dtype': 'float64', 'timezone': 'UTC', 'info': 'the POSIX_timestamp is stored as a 64-bit floating point number, representing the seconds elapsed since 1970-01-01 00:00 UTC time'}}, 'Date_created': '2025-03-01 21:22:42', 'Last_modified': '2025-03-01 21:22:42'}, measurementID='test', dataIn=                        _ Date_Time_GMT_00_00  Temp_C_5cm  Temp_C_25cm  \
2024-07-20 21:00:00     1   24/07/20 21:00:00       7.217        2.423   
2024-07-20 21:30:00     2   24/07/20 21:30:00      11.759        6.712   
2024-07-20 22:00:00     3   24/07/20 22:00:00      11.467        5.257   
2024-07-20 22:30:00     4   24/07/20 22:30:00      11.029        4.428   
2024-07-20 23:00:00     5   24/0

In [53]:

# siteFields = set(self.__dataclass_fields__) - {f for base in siteRecord.__bases__ if hasattr(base,'__dataclass_fields__') for f in base.__dataclass_fields__}
# siteFields = set(self.__dataclass_fields__) - {f for base in origin.__bases__ if hasattr(base,'__dataclass_fields__') for f in base.__dataclass_fields__}


def baseFields(self,repr=True,orderd=True):
    X = (set(self.__dataclass_fields__) - {f for base in type(self).__bases__ if hasattr(base,'__dataclass_fields__') for f,v in base.__dataclass_fields__.items() if v.repr==repr})
    print(X)
    print([k for k in self.__dataclass_fields__ if k in X])


from dataclasses import dataclass, field
@dataclass
class A:
    b: int = field(default=2,repr=False)
    j: int = 1

@dataclass
class B(A):
    x: str = 'a'
    b: int = field(default=3,repr=False)
    y: str = 'a'

baseFields(B())

{'b', 'y', 'x'}
['b', 'x', 'y']
